In [3]:
import json

with open('Musical_Instruments_5.json') as f:
    data = [json.loads(line) for line in f]
data[0]

{'reviewerID': 'A2IBPI20UZIR0U',
 'asin': '1384719342',
 'reviewerName': 'cassandra tu "Yeah, well, that\'s just like, u...',
 'helpful': [0, 0],
 'reviewText': "Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",
 'overall': 5.0,
 'summary': 'good',
 'unixReviewTime': 1393545600,
 'reviewTime': '02 28, 2014'}

In [4]:
len(data)

10261

In [5]:
import pandas as pd

In [6]:
data_v1 = pd.DataFrame(data)

In [7]:
data_v1.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5.0,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5.0,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5.0,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5.0,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5.0,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [12]:
print("ratio of negative to positive is",sum(data_v1["overall"]<=4)/sum(data_v1["overall"]==5))

ratio of negative to positive is 0.47895647160565


In [14]:
review_and_overall_data = data_v1.loc[:, ["overall","reviewText"]] 

In [16]:
def overall_to_sentiment(row):
    if row['overall'] <= 4:
        return 0
    else:
        return 1

review_and_overall_data['sentiment'] = review_and_overall_data.apply(lambda row: overall_to_sentiment(row),axis = 1)

In [17]:
review_and_overall_data.head()

,overall,reviewText,sentiment
0,5.0,"Not much to write about here, but it does exac...",1
1,5.0,The product does exactly as it should and is q...,1
2,5.0,The primary job of this device is to block the...,1
3,5.0,Nice windscreen protects my MXL mic and preven...,1
4,5.0,This pop filter is great. It looks and perform...,1


In [18]:
review_and_overall_data.iloc[0,1]

"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,"

In [19]:
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from nltk.corpus import wordnet
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')


In [20]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [21]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    filtered_tokens = [token for token in filtered_tokens if token.isalpha()]
    filtered_tokens = [wordnet_lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in filtered_tokens if token.isalpha()]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

# lemmatizer = WordNetLemmatizer()
# word = 'feet'
# print(lemmatizer.lemmatize(word, get_wordnet_pos(word)))


In [22]:
review_and_overall_data['reviewText_normal'] = normalize_corpus(review_and_overall_data['reviewText'])

In [23]:
review_and_overall_data.head()

,overall,reviewText,sentiment,reviewText_normal
0,5.0,"Not much to write about here, but it does exac...",1,much write exactly suppose filter pop sound re...
1,5.0,The product does exactly as it should and is q...,1,product exactly quite affordablei realize doub...
2,5.0,The primary job of this device is to block the...,1,primary job device block breath would otherwis...
3,5.0,Nice windscreen protects my MXL mic and preven...,1,nice windscreen protects mxl mic prevents pop ...
4,5.0,This pop filter is great. It looks and perform...,1,pop filter great look performs like studio fil...


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(review_and_overall_data['reviewText_normal'])
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix,2), columns=voc ab)

,aa,aaa,ab,aback,abalone,abalonefender,abandon,abbe,abcd,abehringer,...,zoneso,zoom,zoomnow,zt,ztif,ztupdate,zune,zvex,zylgian,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Xs = tv_matrix
y= review_and_overall_data["overall"]

X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.33, random_state=42)

reg = LinearRegression().fit(X_train,y_train)

#MSEs = cross_val_score(lin_reg, Xs, y, scoring='neg_mean_squared_error', cv=5)

#mean_MSE = np.mean(MSEs)

#print(mean_MSE)



-2.964220112012409
[ 1.92334642 10.18332799 -1.3303786  ...  1.35240497  0.
 -6.91587737]
5.0


In [55]:
print(reg.score(X_train,y_train))
print(reg.coef_)
print(reg.intercept_) 

1.0
[ 1.92334642 10.18332799 -1.3303786  ...  1.35240497  0.
 -6.91587737]
5.0


In [58]:
ypred_reg=reg.predict(X_train)

In [59]:
ypred_reg[0]

2.999999999999999

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge = Ridge()

parameters = {'alpha':[1e-15,1e-10,1e-8,1e-4,1e-4,1e-2,1,5,10,20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)

ridgeReg = ridge_regressor.fit(X_train,y_train)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.31861e-18): result may not be accurate.
  overwrite_a=False)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.08376e-18): result may not be accurate.
  overwrite_a=False)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.90765e-18): result may not be accurate.
  overwrite_a=False)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.45523e-18): result may not be accurate.
  overwrite_a=False)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.72588e-18): result may not be accurate.
  overwrite_a=False)


In [65]:
ypred_rid=ridgeReg.predict(X_train)
ypred_rid[0]
print(ridgeReg.score(X_train,y_train))

-0.30487173778233595


In [88]:
# compare
ypred_reg_test=reg.predict(X_test)
ypred_rid_test=ridgeReg.predict(X_test)
result=pd.DataFrame({'linreg':ypred_reg_test,'ridreg':ypred_rid_test,'y':y_test})

In [89]:
result['linreg1'] = round(result['linreg'])
result['linreg1'] = result['linreg1'].clip(lower=0,upper=5)
result['ridreg1'] = round(result['ridreg'])
result['ridreg1'] = result['ridreg1'].clip(lower=0,upper=5)


In [97]:
print(np.mean(np.sum((result['linreg1']-result['y']) ** 2)))
print(np.mean(np.sum((result['ridreg1']-result['y']) ** 2)))
print(np.sum(result['linreg1']==result['y'])/result.shape[0])
print(np.sum(result['ridreg1']==result['y'])/result.shape[0])
print(np.sum(result['y']==5)/result.shape[0])

13609.0
2477.0
0.3979923235901978
0.5544729849424269
0.673752583407145


In [86]:
result['linreg1'].clip(lower=0,upper=5)

2507     4.0
5159     4.0
932     -0.0
1190     1.0
2619     1.0
251      1.0
2487    -0.0
9920     2.0
7835     5.0
7260     5.0
6177     5.0
6070     5.0
1617     5.0
2273     2.0
8444     4.0
9364     0.0
1691     5.0
4007     3.0
33       4.0
9288     2.0
2114     1.0
10076    4.0
10260    0.0
3133     5.0
10220    5.0
4898     2.0
6208     1.0
321      5.0
9156     5.0
107      2.0
        ... 
4151     3.0
3091     5.0
2726     5.0
477      3.0
1329     2.0
2283     1.0
9895     5.0
4838     5.0
1777     3.0
6451     5.0
751      3.0
7310     4.0
4304     5.0
7557     5.0
6382     1.0
9547     5.0
8602     5.0
1031     5.0
5501     5.0
5457     1.0
468      5.0
1545     2.0
6682     5.0
2683     4.0
7597     3.0
6545     5.0
1025     2.0
9744     5.0
4137     0.0
6877     2.0
Name: linreg1, Length: 3387, dtype: float64

In [99]:
review_and_overall_data['reviewText_normal'][0]

'much write exactly suppose filter pop sound recording much crisp one low price pop filter amazon might well buy honestly work despite pricing'

In [100]:
print(vocab)

['aa', 'aaa', 'ab', 'aback', 'abalone', 'abalonefender', 'abandon', 'abbe', 'abcd', 'abehringer', 'abelton', 'abercrombie', 'abhorrent', 'abide', 'ability', 'abing', 'abit', 'abitity', 'able', 'ableto', 'ableton', 'abnormal', 'abnormality', 'abo', 'abobe', 'abomination', 'abotu', 'abound', 'abouthello', 'aboutits', 'aboutmy', 'aboutthis', 'aboutturns', 'aboutupdate', 'aboutwindtech', 'abovec', 'abovementioned', 'aboveplanet', 'abrams', 'abrasion', 'abrasive', 'abrasiveif', 'abroad', 'abrupt', 'absence', 'absent', 'abslutely', 'absolute', 'absolutely', 'absolutelyif', 'absolutley', 'absolutly', 'absorb', 'absorbed', 'absorbs', 'absurd', 'absurdly', 'abundance', 'abundant', 'abuse', 'abusethe', 'abusive', 'abut', 'aby', 'abysmal', 'ac', 'acadapter', 'acc', 'accc', 'accent', 'accentuate', 'accentuates', 'accepeted', 'accept', 'acceptable', 'acceptablein', 'acceptableover', 'accepts', 'accesible', 'accesories', 'access', 'accessary', 'accessibility', 'accessible', 'accessibly', 'accessorie